In [1]:
import os
import numpy as np
import numpy.linalg as la
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt

In [8]:
from model import Net
from tdnn import TDNN
from dataloader import DataLoader

In [9]:
device = torch.device("cuda:0")
torch.cuda.is_available()

True

In [10]:
file_name = 'mfcc'

train_path = 'mfcc'
y_train = np.load(train_path + '/speakers.npy')
train_size = y_train.shape[0]
print(train_size)

test_path = 'mfcc_test'
y_test = np.load(test_path + '/speakers.npy')
test_size = y_test.shape[0]
print(test_size)

model = torch.load('models/model_acc90.pkl').to(device)

1092008
36237


In [21]:
embeddings_train = np.zeros((train_size, 512))
for i in range(train_size):
    print(f'\r{i}/{train_size}', end='')
    mfcc = torch.load(train_path + '/' + file_name + str(i)).to(device)
    emb = model.embedding(mfcc).cpu().detach().numpy().squeeze(0)
    embeddings_train[i] = emb
np.save('embeddings_train.npy', embeddings_train)

1092007/1092008

In [22]:
embeddings_test = np.zeros((test_size, 512))
for i in range(test_size):
    print(f'\r{i}/{test_size}', end='')
    mfcc = torch.load(test_path + '/' + file_name + str(i)).to(device)
    emb = model.embedding(mfcc).cpu().detach().numpy().squeeze(0)
    embeddings_test[i] = emb
np.save('embeddings_test.npy', embeddings_test)

36236/36237

In [24]:
embeddings_train = np.load('embeddings_train.npy')
embeddings_test = np.load('embeddings_test.npy')

### Simple cosine dist

In [28]:
threshold = 0.742
n_same = 0
k_same = 0
n_diff = 0
k_diff = 0
ITERATIONS = 1000000
for i in range(ITERATIONS):
    print(f'\r{i}/{ITERATIONS}', end='')
    rand_idx = np.random.choice(test_size, size=2, replace=True)
    same = (y_test[rand_idx[0]] == y_test[rand_idx[1]])
    pair = embeddings_test[[rand_idx[0], rand_idx[1]]]
    cos_dist = pair[0] @ pair[1] / (la.norm(pair[0]) * la.norm(pair[1]))
    if cos_dist > threshold:
        if same:
            n_same += 1
        else:
            k_diff += 1
            n_diff += 1
    else:
        if same:
            k_same += 1
            n_same += 1
        else:
            n_diff += 1

print(f'\rTotal same pairs: {n_same:<8}  err rate: {k_same/n_same}')
print(f'Total diff pairs: {n_diff:<8}  err rate: {k_diff/n_diff}')

Total same pairs: 10556     err rate: 0.10591133004926108
Total diff pairs: 989444    err rate: 0.1083386224991005


### PLDA

In [30]:
from plda.model import Model
plda = Model(embeddings_train, y_train, n_principal_components=100)
transformed_test = plda.transform(embeddings_test, from_space='D', to_space='U_model')

In [49]:
threshold = -0.55
n_same = 0
k_same = 0
n_diff = 0
k_diff = 0
ITERATIONS = 1000000
for i in range(ITERATIONS):
    print(f'\r{i}/{ITERATIONS}', end='')
    rand_idx = np.random.choice(test_size, size=2, replace=True)
    same = (y_test[rand_idx[0]] == y_test[rand_idx[1]])
    pair = transformed_test[[rand_idx[0], rand_idx[1]]]
    plda_score = plda.calc_same_diff_log_likelihood_ratio(pair[0][None,:], pair[1][None,:])
    if plda_score > threshold:
        if same:
            n_same += 1
        else:
            k_diff += 1
            n_diff += 1
    else:
        if same:
            k_same += 1
            n_same += 1
        else:
            n_diff += 1

print(f'\rTotal same pairs: {n_same:<8}  err rate: {k_same/n_same}')
print(f'Total diff pairs: {n_diff:<8}  err rate: {k_diff/n_diff}')

Total same pairs: 10748     err rate: 0.07713062895422404
Total diff pairs: 989252    err rate: 0.07711483019493516
